In [1]:
import pandas as pd
import numpy as np

#Загрузка данных
finance = pd.read_csv('office_data/financial_data.csv')
prolong = pd.read_csv('office_data/prolongations.csv')

#Смотрим на данные в таблице
finance.info()
prolong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             451 non-null    int64 
 1   Причина дубля  301 non-null    object
 2   Ноябрь 2022    156 non-null    object
 3   Декабрь 2022   159 non-null    object
 4   Январь 2023    139 non-null    object
 5   Февраль 2023   145 non-null    object
 6   Март 2023      168 non-null    object
 7   Апрель 2023    174 non-null    object
 8   Май 2023       190 non-null    object
 9   Июнь 2023      190 non-null    object
 10  Июль 2023      195 non-null    object
 11  Август 2023    199 non-null    object
 12  Сентябрь 2023  186 non-null    object
 13  Октябрь 2023   182 non-null    object
 14  Ноябрь 2023    171 non-null    object
 15  Декабрь 2023   146 non-null    object
 16  Январь 2024    95 non-null     object
 17  Февраль 2024   101 non-null    object
 18  Account        451 non-null   

In [2]:
#Подгатавливаем данные:
prolong['month'] = [s.lower() if isinstance(s, str) else s for s in prolong['month']]
finance.columns = finance.columns.str.lower()

months = [c for c in finance.columns if any(m in c.lower() for m in [
    'янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'
])] #Выделяем список месяцев

#Переводим значения отгрузки в числовой формат
for m in months:
    finance[m] = (
        finance[m]
        .astype(str)
        .str.strip()
        .str.replace(',', '.', regex=False)   # заменяем запятые на точки
        .str.replace(' ', '', regex=False)    # убираем пробелы и неразрывные пробелы
        .str.replace('\u00A0', '', regex=False)
    )
finance[months] = finance[months].replace({'в ноль': 0, 'В ноль': 0, 'стоп': -1 , 'end': -1, 'Стоп': -1 , 'End': -1})    
finance[months] = finance[months].apply(pd.to_numeric, errors='coerce')

#Объединяем таблицы по id
data = finance.merge(prolong, on = 'id', how = 'left')

print(data.shape) #Проверка размера таблицы до

#Проверка на  раннее окончание проекта
def ended_early(row):
    end_month = row['month']
    if end_month not in months:
        return False  # если вдруг месяц не найден
    end_index = months.index(end_month)
    # Проверяем все значения до (и включая) месяца завершения
    return (row[months[:end_index+1]] == -1).any()


#Применяем функцию к каждой строке таблицы
mask = data.apply(ended_early, axis=1)

#Убираем проекты, которые завершились раньше
data = data[~mask]
print(data.shape) #Проверка размера после


(726, 21)
(665, 21)


In [5]:
#Рассчёт коэффициентов

results = []  #все коэффициенты

#Перебираем месяцы, начиная со второго
for i in range(1, len(months)):
    if '2023' in months[i]:
        target_month = months[i]
        prev_month_1 = months[i - 1]
        
        #Рассчёт по менеджерам
        for am in data['AM'].unique():
            am_data = data[data['AM'] == am]
    
            #К1 - Коэффициент пролонгации первого месяца
            ended_prev = am_data[am_data['month'] == prev_month_1]
            
            sum_last = ended_prev[prev_month_1].sum()
            sum_next = ended_prev[target_month].sum()
            K1 = sum_next / sum_last if sum_last > 0 else 0
          
            #К2 -Коэффициент пролонгации второго месяца
            if i >= 2:
                prev_month_2 = months[i - 2]
                ended_prev2 = am_data[am_data['month'] == prev_month_2]
                #Исключаем те, у кого была отгрузка в промежуточный месяц
                ended_prev2 = ended_prev2[(ended_prev2[prev_month_1].isna()) | (ended_prev2[prev_month_1] == 0)]
                sum_last2 = ended_prev2[prev_month_2].sum()
                sum_next2 = ended_prev2[target_month].sum()
                K2 = sum_next2 / sum_last2 if sum_last2 > 0 else 0
            else:
                K2 = None  #Для первых месяцев невозможно посчитать K2
            results.append({
                'Месяц пролонгации': target_month,
                'Менеджер': am,
                'K1 (первый месяц)': K1,
                'K2 (второй месяц)': K2
            })

#Преобразуем в таблицу
coef_df = pd.DataFrame(results)
print(coef_df.head(10))

  Месяц пролонгации                       Менеджер  K1 (первый месяц)  \
0       январь 2023   Васильев Артем Александрович           0.556236   
1       январь 2023  Соколова Анастасия Викторовна           0.427332   
2       январь 2023      Михайлов Андрей Сергеевич           0.687923   
3       январь 2023    Попова Екатерина Николаевна           0.440678   
4       январь 2023        Иванова Мария Сергеевна           0.270066   
5       январь 2023    Смирнова Ольга Владимировна           0.743395   
6       январь 2023       Кузнецов Михаил Иванович           1.265979   
7       январь 2023                        без А/М           0.000000   
8       январь 2023        Петрова Анна Дмитриевна           0.000000   
9       январь 2023     Федорова Марина Васильевна           0.000000   

   K2 (второй месяц)  
0           0.000000  
1           0.710255  
2           0.000000  
3           0.000000  
4           0.000000  
5           0.000000  
6           0.000000  
7           

In [6]:
#Рассчёт средних коэффициентов по отделу (можно сделать взвешенно при необходимости)
dept_summary = (
    coef_df.groupby('Месяц пролонгации')[['K1 (первый месяц)', 'K2 (второй месяц)']]
    .mean()
    .reset_index()
)
dept_summary['Менеджер'] = 'Отдел в целом'
dept_summary.sort_values('Месяц пролонгации')
#Объединяем
final_df = pd.concat([coef_df, dept_summary], ignore_index=True)

#Сортируем месяцы в алфавитном порядке
final_df['Месяц пролонгации'] = pd.Categorical(
    final_df['Месяц пролонгации'],
    categories=months,
    ordered=True
)

final_df = final_df.sort_values('Месяц пролонгации')

print(final_df.head(10)) #Оцениваем результат

    Месяц пролонгации                       Менеджер  K1 (первый месяц)  \
0         январь 2023   Васильев Артем Александрович           0.556236   
10        январь 2023                            NaN           0.000000   
9         январь 2023     Федорова Марина Васильевна           0.000000   
8         январь 2023        Петрова Анна Дмитриевна           0.000000   
7         январь 2023                        без А/М           0.000000   
6         январь 2023       Кузнецов Михаил Иванович           1.265979   
143       январь 2023                  Отдел в целом           0.399237   
4         январь 2023        Иванова Мария Сергеевна           0.270066   
1         январь 2023  Соколова Анастасия Викторовна           0.427332   
5         январь 2023    Смирнова Ольга Владимировна           0.743395   

     K2 (второй месяц)  
0             0.000000  
10            0.000000  
9             0.000000  
8             0.000000  
7             0.000000  
6             0.000000  

In [11]:
#Записываем в файл результат
final_df.to_excel('prolongation_coefficients_2023.xlsx', index=False)